### Variables dicotómicas

In [9]:
%pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [10]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns


In [ ]:
train_df = pd.read_csv("data/train.csv")
test_df = pd.read_csv("data/test.csv")

train_df.head()


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [12]:
bins = [0, 130000, 200000, float('inf')]
labels = ['Barata', 'Media', 'Cara']
train_df['PrecioCategoria'] = pd.cut(train_df['SalePrice'], bins=bins, labels=labels)


In [13]:
# Crear variable categórica de precio
bins = [0, 130000, 200000, float('inf')]
labels = ['Barata', 'Media', 'Cara']
train_df['PrecioCategoria'] = pd.cut(train_df['SalePrice'], bins=bins, labels=labels)

# Crear variables dicotómicas
train_df['Es_Barata'] = (train_df['PrecioCategoria'] == 'Barata').astype(int)
train_df['Es_Media'] = (train_df['PrecioCategoria'] == 'Media').astype(int)
train_df['Es_Cara'] = (train_df['PrecioCategoria'] == 'Cara').astype(int)

# Verificar resultado
train_df[['SalePrice', 'PrecioCategoria', 'Es_Barata', 'Es_Media', 'Es_Cara']].head(10)


,SalePrice,PrecioCategoria,Es_Barata,Es_Media,Es_Cara
0,208500,Cara,0,0,1
1,181500,Media,0,1,0
2,223500,Cara,0,0,1
3,140000,Media,0,1,0
4,250000,Cara,0,0,1
5,143000,Media,0,1,0
6,307000,Cara,0,0,1
7,200000,Media,0,1,0
8,129900,Barata,1,0,0
9,118000,Barata,1,0,0


### Punto 1 – Clasificación y codificación de precios

Se ha creado una nueva variable categórica `PrecioCategoria` que agrupa las viviendas en tres rangos de precio:

- **Barata**: hasta \$130,000
- **Media**: entre \$130,001 y \$200,000
- **Cara**: más de \$200,000

Con esta categorización, se generaron tres columnas dicotómicas (`Es_Barata`, `Es_Media`, `Es_Cara`) que tienen valores 0 o 1, lo cual facilita su uso como variable dependiente en modelos binarios como la regresión logística.


In [14]:
# Eliminar columnas no útiles o objetivo
excluded_columns = [
    'Id', 'SalePrice', 'PrecioCategoria', 'Es_Barata', 'Es_Media', 'Es_Cara'
]

# Seleccionar solo variables numéricas como features
numeric_cols = train_df.select_dtypes(include=['int64', 'float64']).columns
features = [col for col in numeric_cols if col not in excluded_columns]

# Variables finales para modelado
X = train_df[features]
y = train_df['Es_Cara']


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42
)

print(f"Train: {X_train.shape}, Test: {X_test.shape}")


Train: (1095, 36), Test: (365, 36)


### Punto 2 – División del conjunto de datos

Para mantener la reproducibilidad y la consistencia con hojas anteriores, se usó `train_test_split` con `random_state=42`, dividiendo los datos en:

- 75% para entrenamiento (`X_train`)
- 25% para prueba (`X_test`)

En esta etapa, únicamente se utilizó la variable binaria `Es_Cara` como objetivo para entrenar modelos que predigan si una casa es cara o no.
